In [12]:
import sys

sys.path.append("../")

import MetaTrader5 as mt5
import pytz
import numpy as np

from datetime import datetime, timedelta
from terminal.Extensions.MT5Ext import MT5Ext

MT5Ext.initialize()

In [13]:
from_date = datetime(2023, 1, 1, hour=6, tzinfo=pytz.utc)
to_date = datetime(2024, 1, 1, tzinfo=pytz.utc)
date = None
data_end = None

trades_list = None
symbol = "WIN$N"

while from_date < to_date:
    date = from_date
    date_end = from_date.replace(hour=23)
    trades = mt5.copy_ticks_range(
        symbol, date, date_end, mt5.COPY_TICKS_TRADE
    )
    if mt5.last_error()[0] == mt5.RES_S_OK and len(trades.shape) == 1:
        if trades_list is None:
            trades_list = trades
        else:
            trades_list = np.append(trades_list, trades)
    from_date += timedelta(days=1)
    del trades

    if date.month != from_date.month:
        np.savez_compressed(
            f"ticks_{symbol}_{date.year}_{date.month}.npz", ticks=trades_list
        )
        trades_list = None

if trades_list is not None:
    np.savez_compressed(
        f"ticks_{symbol}_{date.year}_{date.month}.npz", ticks=trades_list
    )
    del trades_list